# ODA data by Sector (with donor and recipient)
This tutorial shows you how to build a Pandas DataFrame containing ODA data from any number of donors to any number of recipients, across sectors available in the list of indicators, over a range of years.

The tutorial has the following steps:
1. Import `ODAData`, the main tool used to interact with the data
2. Create an instance of `ODAData` with the specific arguments you would like to use
3. Load the indicators and get the DataFrame
4. Optionally, export the DataFrame

## 1. Import ODAData

We can gather the data we need using the `ODAData` class. An object from this class can:
    - Get data for specific indicators
    - Optionally, filter the data for specific donors, recipients and years
    - Optionally, exchange and deflate data

The `oda_data` package automatically downloads the relevant datasets (i.e. DAC1, DAC2a or CRS tables) depending on the indicators selected in step 3.

It is highly recommended that you specify the folder where you want to store, and from where you want to read, this raw data. This must be done once in each notebook or script where we use the `oda_data` package. You can specify the data path if the raw data has already been downloaded, or if you haven't yet specified the download path in your notebook and script.

In [ ]:
from oda_data import ODAData, set_data_path

# in section 3, we give the option to retreive data for specific donor and recipient groupings, so we need to import the following
from oda_data.tools.groupings import donor_groupings, recipient_groupings

# If you haven't set the data path, you can do it now.
set_data_path(path="../tutorials/data")

# 2. Create an instance of ODAData

Next, we need to set the right arguments which are used to create an instance of ODAData to produce the right DataFrame. The arguments required are:
- *years*: you must specify the `years`, as an `int`, `list` or `range`
- *donors*: you can _optionally_ specify the `donors` you want the output to have (as `int`, or `list`) of donor codes.
- *recipients*: you can _optionally_ specify the `recipients` you want the output to have. Not all indicators
    need or accept recipients. If using an indicator for which recipients are not an option, a warning will be logged to
    the console and the recipients ignored for that indicator.
- *currency*: you can _optionally_ specify the `currency` in which you want your data to be shown. If not specified,
    by default, `USD` will be used. Other options include `EUR`, `GBP` and `CAN`.
- *prices*: you can _optionally_ specify the `prices` in which you want your data to be shown. If not specified,
    by default, `current` will be used. The other option is `constant`. If specifying `constant` a `base_year` must be set.
- *base_year*: you must specify a `base_year` if you have set `prices = 'constant'`. If you have chosen `current` prices,
    by default, `base_year` will be `None`.

You can use a few methods provided by the ODAData object to see the available donors, recipients and donor/recipient groupings.

In [ ]:
# print a list of available donors
ODAData().available_donors()

In [ ]:
# print a list of available recipients
ODAData().available_recipients()

In [ ]:
# Print available donor groupings
print(donor_groupings().keys())

In [ ]:
# Print available recipient groupings
print(recipient_groupings().keys())

Next, in order to get the data we want, we will create an instance of the `ODAData` class by specifying the correct arguments.

We will store this instance in a variable called `oda`, which we will use later to load the indicators and get the DataFrame we're after.

Below are some example settings for this tutorial. For clarity, we will first store them in variables, but you can
always pass them directly as arguments to the `ODAData` class.

In [ ]:
# Select years as (for example) a range. Remember ranges are exclusive of the upper bound.
years = range(2018, 2021)

# Select donors, which must be specified by their codes. To get all donors, do not use this argument.
donors = [4, 5, 12, 302]

# Select recipients, which must be specified by their codes or available country groupings. In this example, we are selecting LDCs (which we define below).
recipients = list(recipient_groupings()['ldc_countries'])

# Set the base year. We must set this given that we've asked for constant data.
base_year = 2021

# Instantiate the `ODAData` class and store it in a variable called 'oda'
oda = ODAData(years=years,
              donors=donors,
              recipients=recipients)

## 3. Load the indicators and get the DataFrame

Then we tell `oda` to load the indicator(s) that are useful for our analysis. For this tutorial, we will use the *"recipient_bilateral_flow_disbursement_gross"*, and *"imputed_multi_flow_disbursement_gross"* indicators

A full list of indicators can be seen by using the `.available_indicators()` method.

In [ ]:
# print available indicators
ODAData().available_indicators()

In [ ]:
# Create a variable with the list of indicators for this analysis.
# The indicator can also be directly passed as an argument in the step below.
indicators = ['crs_bilateral_flow_disbursement_gross',
              'imputed_multi_flow_disbursement_gross']

# Add all the indicators in our `indicators` list
for indicator in indicators:
    oda.load_indicator(indicator)

# Get a DataFrame with all the data.
df = oda.get_data('all')

# # Finally, group the DataFrame rows by year, currency, prices and indicator
df = df.groupby(['year','donor_code', "recipient_code", "indicator", 'purpose_code', 'currency', 'prices'], observed=True,
           dropna=False)['value'].sum(numeric_only=True).reset_index(drop=False)

# show the resulting dataframe
df.sample(10)

## 4. Optionally export DataFrame as CSV

Finally, we can export the DataFrame as a CSV if required.

In [ ]:
df.to_csv(r'../tutorials/output/oda_by_sectors_oda.csv', index=False)